<a href="https://colab.research.google.com/github/charliebrown314/NLP_Grammar/blob/main/Grammar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy
import nltk
from nltk import grammar, parse, Expression

## Model

In [ ]:
#Define model

# Mostly Joe, some Jess #

v = """
Tom => t
Robin => r
Mia => mia
Sam => s
Jack => j
Andrew => a
Dave => d
Jess => je
Mike => m
Pumpkin => p
#This is my cats name so thats why its a proper name :) 
Kate => k
Lance => l
Sheba => sh
Bella => b
Booker => bo
Leon => le
Claire => c
Joe => jo
Felix => fefe
Tim => t
Mary => ma
Alex => al
unknown => uk
corner => {cor}
A blue Sedan => se
A black Sedan => bs
The dealership => deal
sedan => {se, bs}
blue => {se}
car => {se, bs}
customer => {cu,st}
cat => {p}
tall => {j}
skinny => {j}
man => {j,d,l,bo,jo,t}
boy => {s,a,m,le,f,al}
woman => {mia,r,k}
girl => {b,c,girl}
sailor => {j}
restaurant => {re}
leave => {(j,re),(l,re),(m,re),(t,re),(cu,deal)}
sneeze => {l,m,j}
interrupt => {(r,sail)(j,r)}
moustache => {mou}
see => {(t,p)(mia,p),(ma,fefe),(t,se)}
around => {(deal,cor)}
buy => {(s,se,le),(s,bs),(s,se),(r,se)}
in => {(bs,deal)}
dealership => {deal}
black => {bs}
brochure => {br}
insult => {(cu,uk)}
give => {(mia,br,cu)}
drunk => {j}
with => {(j,mou)}
"""
val = nltk.Valuation.fromstring(v)

g = nltk.Assignment(val.domain)

m = nltk.Model(val.domain, val)

## Grammar

In [ ]:
# Define grammar

# Mostly Jess, Some Joe#

gs = r"""

% start S
############################
# Grammar Rules
#############################
S[SEM = <?s>] -> WHP S[SEM=?s]/NP
S[SEM = <?v(x)>] -> WHP VP[SEM = ?v]/NP
S[SEM = <?v(x)>] -> WHP VP[SEM = ?v]

TV[VC=pass,SEM=<(\R Q z.Q(\w. (((R(\T.T))(w))(z))))(?tv)>] -> TV[VC=act,SEM=?tv]
# DTV[VC=pass,SEM=<(\R Q z y.Q(\w. (((R(\T.T))(w))(y)(z))))(?dtv)>] -> DTV[VC=act,SEM=?dtv]


# wh-questions with gaps
S[SEM = <?subj(?vp)>]/NP -> AUX NP[SEM=?subj] VP[SEM=?vp]/NP
S[SEM = <?subj(?vp)>] -> AUX NP[SEM=?subj] VP[SEM=?vp]
S[SEM = <?subj(?vp)>] -> NP[SEM=?subj] VP[SEM=?vp]
S[SEM = <\x.(?subj(?vp(x)))>]/NP -> NP[SEM =?subj] VP[SEM=?vp]/NP

VP[VC=act,SEM=<?v(?obj)>] -> TV[VC=act,SEM=?v] NP[SEM=?obj]
VP[VC=pass,SEM=<?v(?obj)>] -> TV[VC=pass,SEM=?v] PP[PFORM=passive_by,SEM=?obj]
VP[VC=pass,SEM=<?v(?obj)>] -> TV[VC=pass,SEM=?v] PP[PFORM=dative_to,SEM=?obj]
VP[VC=pass,SEM=<?v(?obj)>] -> TV[VC=pass,SEM=?v] PP[PFORM=for,SEM=?obj]


VP[SEM=<?vp>] -> BE VP[VC=pass,SEM=?vp]
VP[SEM=?v] -> IV[SEM=?v]
VP[SEM=<?v(?obj,?pp)>] -> DTV[SEM=?v] NP[SEM=?obj] PP[SEM=?pp]
VP[SEM=<?v(?obj,\P.P(w))>]/NP -> DTV[SEM=?v] NP[SEM=?obj] PP[PFORM=passive_by]/NP
VP[SEM=<?v(?obj,\P.P(w))>]/NP -> DTV[SEM=?v] NP[SEM=?obj] PP[PFORM=for]/NP
VP[SEM=<\x.(?v(?obj,\P.P(x)))>]/NP -> DTV[SEM=?v] NP[SEM=?obj] PP[PFORM=dative_to]/NP
VP[SEM=<?v(?obj)>] -> TV[SEM=?v] NP[SEM=?obj]
VP[SEM=<(\R Q z.Q(\w. (((R(\T.T))(w))(z))))(?v)(\P.P(w))>]/NP -> TV[VC=act,SEM=?v] PP[PFORM=passive_by]/NP

VP[SEM=<?v(\P.P(w))>]/NP -> TV[SEM=?v,VC=act]
VP[SEM=<?v(\P.P(w))>]/NP -> TV[SEM=?v,VC=pass] PP[PFORM=passive_by]/NP

PP[PFORM=n_a,SEM=<\x.?p(?np, x)>] -> P[PFORM=n_a,SEM=?p]  NP[SEM=?np]
PP[PFORM=passive_by,SEM=?np] -> P[PFORM=passive_by]  NP[SEM=?np]

PP[PFORM=dative_to,SEM=?np] -> P[PFORM=dative_to] NP[SEM=?np]
PP[PFORM=dative_to,SEM=?np] -> P[PFORM=for] NP[SEM=?np]
PP[PFORM=dative_to]/NP -> P[PFORM=dative_to]
PP[PFORM=for]/NP -> P[PFORM=for]
PP[PFORM=passive_by]/NP -> P[PFORM=passive_by]


NP[SEM=<?det(?nom)>] -> DT[SEM=?det]  N[SEM=?nom]
NP[SEM=?np] -> PN[SEM=?np]
NP[SEM=?np] -> NP[SEM=?np] CP NP VP/NP


ADJ[SEM=?jj] -> ADJ[SEM=?jj] ADJ[SEM=?jj]
N[SEM=<?jj(?nom)>] -> ADJ[SEM=?jj] N[SEM=?nom]
N[SEM=<?pp(?nom)>] -> N[SEM=?nom] PP[PFORM=n_a,SEM=?pp]

CP[SEM=<?s>] -> C S[SEM = ?s]/NP
CP[SEM=<?vp>] -> C VP[SEM=?vp]
N[SEM=<\x.(?nom(x) & ?cp(x))>] -> N[SEM=?nom] CP[SEM=?cp]
N[SEM=<\x.(?nom(x) & ?cp(x))>] -> N[SEM=?nom] CP[SEM=?cp]/NP

#############################
# Lexical entries
#############################

AUX -> 'do'
AUX -> 'should'
AUX -> 'be'

WHP -> 'which'
WHP -> 'who'
WHP -> 'what'
WHP -> 'where'

C -> 'that'

FOR -> 'for'
BE -> 'be' 
P[PFORM=passive_by] -> 'by'


P[PFORM=n_a, SEM=<\P Q x.(Q(x) & P(\y.with(x,y)))>] -> 'with'
P[PFORM=n_a, SEM=<\P Q x.(Q(x) & P(\y.around(x,y)))>] -> 'around'
P[PFORM=n_a, SEM=<\P Q x.(Q(x) & P(\y.in(x,y)))>] -> 'in'
P[PFORM=n_a, SEM=<\P Q x.(Q(x) & P(\y.near(x,y)))>] -> 'near'
P[PFORM=n_a, SEM=<\P Q x.(Q(x) & P(\y.under(x,y)))>] -> 'under'
P[PFORM=n_a, SEM=<\P Q x.(Q(x) & P(\y.over(x,y)))>] -> 'over'
P[PFORM=n_a, SEM=<\P Q x.(Q(x) & P(\y.on(x,y)))>] -> 'on'

P[PFORM=dative_to] -> 'to'
P[PFORM=for] -> 'for'
# P[PFORM=for] -> 'with'


DT[SEM=<\P Q.all x.(P(x) -> Q(x))>] -> 'every'
DT[SEM=<\P Q.all x.(P(x) -> Q(x))>] -> 'all'
DT[SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'some'
DT[SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'a'
DT[SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'an'
DT[SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'the'


ADJ[SEM=<\P x.(P(x) & tall(x))>] -> 'tall'
ADJ[SEM=<\P x.(P(x) & skinny(x))>] -> 'skinny'
ADJ[SEM=<\P x.(P(x) & black(x))>] -> 'black'
ADJ[SEM=<\P x.(P(x) & drunk(x))>] -> 'drunk'
ADJ[SEM=<\P x.(P(x) & fat(x))>] -> 'fat'
ADJ[SEM=<\P x.(P(x) & blue(x))>] -> 'blue'
ADJ[SEM=<\P x.(P(x) & stinky(x))>] -> 'stinky'
ADJ[SEM=<\P x.(P(x) & sad(x))>] -> 'sad'
ADJ[SEM=<\P x.(P(x) & lovely(x))>] -> 'lovely'
ADJ[SEM=<\P x.(P(x) & polite(x))>] -> 'polite'
ADJ[SEM=<\P x.(P(x) & adaptable(x))>] -> 'adaptable'
ADJ[SEM=<\P x.(P(x) & scary(x))>] -> 'scary'
ADJ[SEM=<\P x.(P(x) & dazzling(x))>] -> 'dazzling'
ADJ[SEM=<\P x.(P(x) & cold(x))>] -> 'cold'

PN[SEM=<\P.P(Robin)>] -> 'robin'
PN[SEM=<\P.P(Mia)>] -> 'mia'
PN[SEM=<\P.P(Sam)>] -> 'sam'
PN[SEM=<\P.P(Tom)>] -> 'tom'
PN[SEM=<\P.P(Jack)>] -> 'jack'
PN[SEM=<\P.P(Andrew)>] -> 'andrew'
PN[SEM=<\P.P(Dave)>] -> 'dave'
PN[SEM=<\P.P(Jess)>] -> 'jess'
PN[SEM=<\P.P(Mike)>] -> 'mike'
PN[SEM=<\P.P(Pumpkin)>] -> 'pumpkin'
#This is my cats name so thats why its a proper name :) 
PN[SEM=<\P.P(Kate)>] -> 'kate'
PN[SEM=<\P.P(Lance)>] -> 'lance'
PN[SEM=<\P.P(Sheba)>] -> 'sheba'
PN[SEM=<\P.P(Bella)>] -> 'bella'
PN[SEM=<\P.P(Booker)>] -> 'booker'
PN[SEM=<\P.P(Leon)>] -> 'leon'
PN[SEM=<\P.P(Claire)>] -> 'claire'
PN[SEM=<\P.P(Joe)>] -> 'joe'
PN[SEM=<\P.P(Tim)>] -> 'tim'
PN[SEM=<\P.P(Mary)>] -> 'mary'
PN[SEM=<\P.P(Felix)>] -> 'felix'
PN[SEM=<\P.P(Alex)>] -> 'alex'
PN[SEM=<\P.P(Tamika)>] -> 'tamika'
PN[SEM=<\P.P(Gru)>] -> 'gru'

N[SEM=<\x.man(x)>] -> 'man'
N[SEM=<\x.girl(x)>] -> 'girl'
N[SEM=<\x.boy(x)>] -> 'boy'
N[SEM=<\x.dog(x)>] -> 'dog'
N[SEM=<\x.cat(x)>] -> 'cat'
N[SEM=<\x.sedan(x)>] -> 'sedan'
N[SEM=<\x.brochure(x)>] -> 'brochure'
N[SEM=<\x.sailor(x)>] -> 'sailor'
N[SEM=<\x.moustache(x)>] -> 'moustache'
N[SEM=<\x.customer(x)>] -> 'customer'
N[SEM=<\x.dealership(x)>] -> 'dealership'
N[SEM=<\x.corner(x)>] -> 'corner'
N[SEM=<\x.restaurant(x)>] -> 'restaurant'
N[SEM=<\x.computert(x)>] -> 'computer'
N[SEM=<\x.unicorn(x)>] -> 'unicorn'
N[SEM=<\x.dog(x)>] -> 'dog'
N[SEM=<\x.parrot(x)>] -> 'parrot'
N[SEM=<\x.carrot(x)>] -> 'carrot'
N[SEM=<\x.box(x)>] -> 'box'
N[SEM=<\x.notebook(x)>] -> 'notebook'
N[SEM=<\x.shirt(x)>] -> 'shirt'
N[SEM=<\x.smoothie(x)>] -> 'smoothie'
N[SEM=<\x.book(x)>] -> 'book'
N[SEM=<\x.soup(x)>] -> 'soup'
N[SEM=<\x.bird(x)>] -> 'bird'
N[SEM=<\x.guitar(x)>] -> 'guitar'
N[SEM=<\x.theorbo(x)>] -> 'theorbo' 
N[SEM=<\x.suspect(x)>] -> 'suspect'
N[SEM=<\x.foreigner(x)>] -> 'foreigner'
N[SEM=<\x.coffee(x)>] -> 'coffee'
N[SEM=<\x.tea(x)>] -> 'tea'
N[SEM=<\x.virus(x)>] -> 'virus'
N[SEM=<\x.staircase(x)>] -> 'staircase'
N[SEM=<\x.phone(x)>] -> 'phone'
N[SEM=<\x.lotion(x)>] -> 'lotion'
N[SEM=<\x.woman(x)>] -> 'woman'
N[SEM=<\x.meme(x)>] -> 'meme'
N[SEM=<\x.woman(x)>] -> 'anime'
N[SEM=<\x.music(x)>] -> 'music'
N[SEM=<\x.motorcycle(x)>] -> 'motorcycle'
N[SEM=<\x.blanket(x)>] -> 'blanket'
N[SEM=<\x.camera(x)>] -> 'camera'
N[SEM=<\x.suit(x)>] -> 'suit'
N[SEM=<\x.wig(x)>] -> 'wig'
N[SEM=<\x.camera(x)>] -> 'camera'
N[SEM=<\x.poster(x)>] -> 'poster'
N[SEM=<\x.popcorn(x)>] -> 'popcorn'
N[SEM=<\x.glitter(x)>] -> 'glitter'
N[SEM=<\x.perfume(x)>] -> 'perfume'
N[SEM=<\x.ramen(x)>] -> 'ramen'
N[SEM=<\x.car(x)>] -> 'car'


IV[SEM=<\x.sneeze(x)>] -> 'sneeze'
IV[SEM=<\x.walk(x)>] -> 'walk'
IV[SEM=<\x.leave(x)>] -> 'leave'
IV[SEM=<\x.leave(x)>] -> 'left'
IV[SEM=<\x.laugh(x)>] -> 'laugh'
IV[SEM=<\x.die(x)>] -> 'die'
IV[SEM=<\x.cry(x)>] -> 'cry'
IV[SEM=<\x.sing(x)>] -> 'sing'
IV[SEM=<\x.sleep(x)>] -> 'sleep'
IV[SEM=<\x.sneeze(x)>] -> 'sneeze'
IV[SEM=<\x.lie(x)>] -> 'lie'

TV[VC=act,SEM=<\P x.P(\y.interrupt(x,y))>] -> 'interrupt'
TV[VC=act,SEM=<\P x.P(\y.insult(x,y))>] -> 'insult'
TV[VC=act,SEM=<\P x.P(\y.buy(x,y))>] -> 'buy'
TV[VC=act,SEM=<\P x.P(\y.chase(x,y))>] -> 'chase'
TV[VC=act,SEM=<\P x.P(\y.leave(x,y))>] -> 'leave'
TV[VC=act,SEM=<\P x.P(\y.leave(x,y))>] -> 'left'
#because spacy tokens this as the direction left it has to be separately in the lexical terms
TV[VC=act,SEM=<\P x.P(\y.kiss(x,y))>] -> 'kiss'
TV[VC=act,SEM=<\P x.P(\y.see(x,y))>] -> 'see'
TV[VC=act,SEM=<\P x.P(\y.bite(x,y))>] -> 'bite'
TV[VC=act,SEM=<\P x.P(\y.drink(x,y))>] -> 'drink'
TV[VC=act,SEM=<\P x.P(\y.like(x,y))>] -> 'like'
TV[VC=act,SEM=<\P x.P(\y.eat(x,y))>] -> 'eat'
TV[VC=act,SEM=<\P x.P(\y.wear(x,y))>] -> 'wear'
TV[VC=act,SEM=<\P x.P(\y.sing(x,y))>] -> 'sing'


DTV[VC=act,SEM=<\P Q x.Q(\z.P(\y.give(x,y,z)))>] -> 'give'
DTV[VC=act,SEM=<\P Q x.Q(\z.P(\y.buy(x,y,z)))>] -> 'buy'
DTV[VC=act,SEM=<\P Q x.Q(\z.P(\y.send(x,y,z)))>] -> 'send'
DTV[VC=act,SEM=<\P Q x.Q(\z.P(\y.tell(x,y,z)))>] -> 'tell'
DTV[VC=act,SEM=<\P Q x.Q(\z.P(\y.show(x,y,z)))>] -> 'show'
DTV[VC=act,SEM=<\P Q x.Q(\z.P(\y.promise(x,y,z)))>] -> 'promise'
DTV[VC=act,SEM=<\P Q x.Q(\z.P(\y.build(x,y,z)))>] -> 'build'
DTV[VC=act,SEM=<\P Q x.Q(\z.P(\y.make(x,y,z)))>] -> 'make'
DTV[VC=act,SEM=<\P Q x.Q(\z.P(\y.prescribe(x,y,z)))>] -> 'prescribe'
DTV[VC=act,SEM=<\P Q x.Q(\z.P(\y.sell(x,y,z)))>] -> 'sell'

"""


# Load grammar
grammar2 = nltk.grammar.FeatureGrammar.fromstring(gs)


## Q/A System

In [ ]:
# Joe #
WHP = [
       'who',
       'what',
       'where',
       'which'
]
NoneAnswerDict = {
    'who': 'Nobody',
    'what': 'Nothing',
    'where': 'Nowhere',
    'which': 'None'
}
def isWH(semTree):
  for wh in WHP:
    if(wh in semTree.leaves()):
      return wh
  return False

def evaluateSentence(text1):
  # Jess and Joe 
  text = text1.lower() # makes all lowercase so it can be handled by grammar


  #Lemmatizing
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(text)
  #Joe
  stems = [token.lemma_ if(token.lemma_ != '-PRON-') else token.text for token in doc] #this also tokenizes at same time
  # print([token.pos_ for token in doc])

  stems.pop()
  #removes punctuation; since Rui made requirement that statement has to have it.
  # print(stems)

  # # Select parser
  # parser = nltk.parse.FeatureChartParser(grammar2)

  # # Find all semantic representations for all parses 
  # print("Active:")
  # for tree in parser.parse(stems):
  #     print(tree)
  #     print('\n Meaning: ',tree.label()['SEM'])
  
  sent = ' '.join(stems)
  results = nltk.evaluate_sents([sent], grammar2, m, g)[0]
  for (syntree, semrep, value) in results:
      tree = syntree
      rep = semrep
      truth = value
  
  print(tree) ###### Uncomment to See parse tree. #######
  print(rep) ###### Uncomment to See semantic Meaning of Questions. #######
  # print(truth)
  # Jess and Joe #
  print(text1, end=': ')
  whQ = isWH(tree)
  if "." in text1:
    if(truth == "Undefined"):
      print("Im not sure.")
    else:
      print("I know.") if(truth) else print("I dont think so.")
  elif whQ:
    formula = rep
    freevar = list(formula.free())[0]
    satisfiers = list(m.satisfiers(formula, freevar, g))
    length = len(satisfiers)
    invValNames = {b:a for a,b in val.items() if isinstance(b, str)}
    if(length < 1):
      print(NoneAnswerDict[whQ])
    else:
      if(satisfiers[0] == "uk"):
        print("Sorry, I dont know their name.")
      else:
        for i in range(0,length - 2):
          print(invValNames[satisfiers[i]],end=', ')
        if(length > 1):
          print(invValNames[satisfiers[length - 2]],end=' and ') 
        print(invValNames[satisfiers[length - 1]])
  else:
    if(truth == "Undefined"):
      print("Im not sure.")
    else:
      print("Yes.") if(truth) else print("No.")

## I/O:

In the above cell, uncomment print(rep) to view the semantic meaning of questions that we parsed, and print(tree) to view the parse tree. (Both Marked by Comments to the right of them.)

In [ ]:
evaluateSentence("A tall skinny man sneezed.") #Passed
evaluateSentence("Mia gave a brochure to the customer.") #Passed
evaluateSentence("Did a tall skinny man sneeze?") #Passed
evaluateSentence("Who sneezed?") #Passed
evaluateSentence("Who did Sam buy a black sedan in the dealership around the corner for?") #Passed
evaluateSentence("Who bought a sedan?") #Passed
evaluateSentence("What did Mia see?") #Passed
evaluateSentence("Tom saw a blue car.") #Passed
evaluateSentence("What did Tom see?") #Passed
evaluateSentence("Did Tom see a blue car?") #Passed
evaluateSentence("A man left the restaurant.") #Passed
evaluateSentence("A cat left the restaurant.") #Passed
evaluateSentence("Did a man leave the restaurant?") #Passed
evaluateSentence("Did a cat left the restaurant?") #Passed
evaluateSentence("Who left the restaurant?") #Passed
evaluateSentence("What did Mary see?") #Passed
evaluateSentence("Sam bought a black sedan in the dealership around the corner.") #Passed
evaluateSentence("The customer that Mia gave a brochure to left the dealership.") #Passed
evaluateSentence("A blue car was seen by Tom.") #Passed
evaluateSentence("What was seen by Tom?") #Passed
evaluateSentence("Was a blue car seen by Tom?") #Passed
evaluateSentence("Robin bought a car that was seen by Tom.") #Passed
evaluateSentence("Robin was interrupted by a drunk sailor with a moustache.") #Passed
evaluateSentence("Did Sam buy a black sedan in the dealership around the corner for Mia?") #Passed
evaluateSentence("Did the customer that Mia gave a brochure to leave the dealership?") #Passed
evaluateSentence("Was Robin interrupted by a drunk sailor with a moustache?") #Passed
evaluateSentence("Sam bought a sedan for Robin.") #Passed
evaluateSentence("What did the customer that Mia gave a brochure to leave?") #Passed
evaluateSentence("Who did Sam buy a sedan for?") #Passed
evaluateSentence("What was Robin interrupted by?") #Passed
evaluateSentence("Was Robin interrupted by Joe?") #Passed
evaluateSentence("What was Alex interrupted by?") #Passed
evaluateSentence("Who Did the customer that Mia gave a brochure to insult?") #Passed



(S[SEM=<exists x.(man(x) & skinny(x) & tall(x) & sneeze(x))>]
  (NP[SEM=<\Q.exists x.(man(x) & skinny(x) & tall(x) & Q(x))>]
    (DT[SEM=<\P Q.exists x.(P(x) & Q(x))>] a)
    (N[SEM=<\x.(man(x) & skinny(x) & tall(x))>]
      (ADJ[SEM=<\P x.(P(x) & tall(x))>] tall)
      (N[SEM=<\x.(man(x) & skinny(x))>]
        (ADJ[SEM=<\P x.(P(x) & skinny(x))>] skinny)
        (N[SEM=<\x.man(x)>] man))))
  (VP[SEM=<\x.sneeze(x)>] (IV[SEM=<\x.sneeze(x)>] sneeze)))
exists x.(man(x) & skinny(x) & tall(x) & sneeze(x))
A tall skinny man sneezed.: I know.
